In [3]:
import pandas

In [8]:
# import the data
df_train = pandas.read_json('train.json')
df_test = pandas.read_json('test.json')


# drop the target column from the data and use it for the labels
classification_column_name = 'requester_received_pizza'
train_data = df_train.drop([classification_column_name], axis=1)
train_labels = df_train[classification_column_name]